In [2]:
import boto3
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import re, datetime

pd.set_option('display.max_columns',None)
warnings.filterwarnings('ignore')

In [1]:
pwd

'/root/AWS_data/Data/V1.0'

In [3]:
def extract_date(x):
    x = x.split('Summed.wav')[1]
    match = re.search('\d{4}-\d{2}-\d{2}', x)
    date = datetime.datetime.strptime(match.group(), '%Y-%m-%d').date()
    return date
    
    
def get_transcripts(s3_bucket=None,prefix_source=None,return_json=False,suffix_csv='.csv',date_lookup=None):
    
    s3 = boto3.resource('s3')
    my_bucket_source = s3.Bucket(s3_bucket)
    df_json = pd.DataFrame()
    df_metadata = pd.DataFrame()
    for bucket_object in my_bucket_source.objects.filter(Prefix=prefix_source):
        if str(bucket_object.key).endswith('.json'):
#             if date_lookup:
#                 date_json = extract_date(bucket_object.key)
#                 if date_json != pd.to_datetime(date_lookup):
#                     continue
            path = 's3://{}/{}'.format(bucket_object.bucket_name, bucket_object.key)
            data = pd.read_json(path, lines=True)
            data['sub_bucket'] = prefix_source
            df_json = pd.concat([df_json, data], axis=0)
        elif str(bucket_object.key).endswith(suffix_csv):
            path = 's3://{}/{}'.format(bucket_object.bucket_name, bucket_object.key)
            df_metadata = pd.read_csv(path)
          
    df_json['transcripts'] = df_json['results'].apply(lambda x: x['transcripts'][0]['transcript'])
    df_json = df_json.reset_index(drop=True)
    
    if return_json:
        return df_json, df_metadata
    
    df_json = df_json.drop(columns=['status', 'accountId'])
    df_json["Date_jobName"] = df_json.jobName.apply(lambda x: extract_date(x))
    df_json.jobName = df_json.jobName.apply(lambda x: x.split('_')[1])
    
    if df_metadata.empty:
        df_metadata = None
        print("There is no metadata available with the jsons")
        
    return df_json, df_metadata

In [1]:
source1 = [f'calldata/bmo/2021-08-0{str(i)}' for i in range(2,10)]
source2 = [f'calldata/bmo/2021-08-{str(i)}' for i in range(10,32)]
source3 = [f'calldata/bmo/2020-08-0{str(i)}' for i in range(4,10)]
source4 = [f'calldata/bmo/2020-08-{str(i)}' for i in range(10,32)]
source5 = [f'calldata/bmo/2021-09-0{str(i)}' for i in range(1,10)]
source6 = [f'calldata/bmo/2021-09-{str(i)}' for i in range(10,32)]
source5 = [f'calldata/bmo/2021-10-0{str(i)}' for i in range(1,10)]
source6 = [f'calldata/bmo/2021-10-{str(i)}' for i in range(10,32)]
source7 = [f'calldata/bmo/2020-06-23']
source8 = [f'calldata/bmo/2020-04-30']

source9 = [f'calldata/meijer/2021-09-0{str(i)}' for i in range(1,10)]
source10 = [f'calldata/meijer/2021-09-{str(i)}' for i in range(10,32)]
source11 = [f'calldata/meijer/2021-10-0{str(i)}' for i in range(1,10)]
source12 = [f'calldata/meijer/2021-10-{str(i)}' for i in range(10,32)]

source13 = [f'calldata/cigna/2021-09-0{str(i)}' for i in range(1,10)]
source14 = [f'calldata/cigna/2021-09-{str(i)}' for i in range(10,32)]
source15 = [f'calldata/cigna/2021-10-0{str(i)}' for i in range(1,10)]
source16 = [f'calldata/cigna/2021-10-{str(i)}' for i in range(10,32)]
#source9 = [f'calldata/meijer/2021-0{str(j)}-0{str(i)}' for j in range(5,10) for i in range(1,11)]
#source10 = [f'calldata/meijer/2021-0{str(j)}-{str(i)}' for j in range(5,10) for i in range(10,32)]
# source5 = [f'calldata/bmo/2021-04-30']
# source6 = [f'calldata/bmo/2021-06-23']
source = source1 + source2 + source3 + source4 + source5 + source6 + source7 + source8 + source9 + source10 + source11 + source12 + source13 + source14 + source15 + source16

source1 = [f'calldata/bmo/2021-08-0{str(i)}' for i in range(2,10)]
source2 = [f'calldata/bmo/2021-08-{str(i)}' for i in range(10,32)]
source3 = [f'calldata/bmo/2020-08-0{str(i)}' for i in range(4,10)] 
source4 = [f'calldata/bmo/2020-08-{str(i)}' for i in range(10,32)]
source5 = [f'calldata/meijer/2021-0{str(j)}-0{str(i)}' for j in range(5,9) for i in range(1,10)]
source6 = [f'calldata/meijer/2021-0{str(j)}-{str(i)}' for j in range(5,9) for i in range(10,32)]
# source5 = [f'calldata/bmo/2021-04-30']
# source6 = [f'calldata/bmo/2021-06-23']
source = source1 + source2 + source3 + source4 + source5 + source6

In [10]:
len(source)

31

In [5]:
concat_df_bmo_trans = pd.DataFrame()
concat_df_bmo_meta = pd.DataFrame()
counter = 0
list_of_problematic_jsons = dict()

for prefix_source in source:
    try:
        df_trans, df_meta = get_transcripts(s3_bucket='adl-core-sagemaker-studio', 
                                                prefix_source=prefix_source,
                                                return_json=False,date_lookup='2021-07-02')

        concat_df_bmo_trans = pd.concat([concat_df_bmo_trans, df_trans], axis=0)
        concat_df_bmo_meta = pd.concat([concat_df_bmo_meta, df_meta], axis=0)

        counter += 1
        
    except KeyError:
        list_of_problematic_jsons[prefix_source] = list_of_problematic_jsons.get(prefix_source, counter)
        
    except Exception as es:
        raise(es)

## Directories Not available

In [6]:
list_of_problematic_jsons #these are weekend dates

{'calldata/cigna/2021-09-04': 3,
 'calldata/cigna/2021-09-05': 3,
 'calldata/cigna/2021-09-06': 3,
 'calldata/cigna/2021-09-11': 7,
 'calldata/cigna/2021-09-12': 7,
 'calldata/cigna/2021-09-18': 12,
 'calldata/cigna/2021-09-19': 12,
 'calldata/cigna/2021-09-25': 17,
 'calldata/cigna/2021-09-26': 17,
 'calldata/cigna/2021-09-31': 21}

In [11]:
len(concat_df_bmo_trans)

15265

In [12]:
concat_df_bmo_trans

,jobName,isRedacted,results,sub_bucket,transcripts,Date_jobName
0,7002934538939741288,True,"{'transcripts': [{'transcript': 'Yes. Yes.'}],...",calldata/cigna/2021-09-01,Yes. Yes.,2021-09-07
1,7002935118760326394,True,{'transcripts': [{'transcript': 'Thank you for...,calldata/cigna/2021-09-01,Thank you for calling Cigna employee service C...,2021-09-07
2,7002935170299933960,True,{'transcripts': [{'transcript': 'Thanks for ca...,calldata/cigna/2021-09-01,Thanks for calling Cigna employee Service Cent...,2021-09-07
3,7002935363573462323,True,"{'transcripts': [{'transcript': 'Hello, imagin...",calldata/cigna/2021-09-01,"Hello, imagine.",2021-09-07
4,7002935651336271208,True,{'transcripts': [{'transcript': 'Thank you for...,calldata/cigna/2021-09-01,Thank you for calling Cigna Employee Service C...,2021-09-07
...,...,...,...,...,...,...
628,7013872154195330692,True,{'transcripts': [{'transcript': 'New york Life...,calldata/cigna/2021-09-30,New york Life Group benefits solution. Service...,2021-10-01
629,7013872489202779843,True,{'transcripts': [{'transcript': 'Thank you for...,calldata/cigna/2021-09-30,Thank you for calling Cigna employee service C...,2021-10-01
630,7013874537902180453,True,{'transcripts': [{'transcript': 'Thank you for...,calldata/cigna/2021-09-30,Thank you for calling Cigna employee service c...,2021-10-01
631,7013876212939426218,True,{'transcripts': [{'transcript': 'Thank you for...,calldata/cigna/2021-09-30,Thank you for calling Cigna employee service c...,2021-10-01


In [7]:
len(concat_df_bmo_meta)

15328

In [8]:
concat_df_bmo_meta.shape

(15328, 37)

In [9]:
concat_df_bmo_meta.columns

Index(['Path', 'File Name', 'Status', 'Segment ID', 'Segment Start Time ',
       'Segment Stop Time', 'Internal Segment Client Start Time',
       'Internal Segment Client Stop Time', 'Participant Station',
       'Segment UCID', 'Participant Trunk Group', 'Participant Trunk Number',
       'Segment Dialed Number', 'Participant Phone-Number',
       'Segment Call Direction Type ID', 'Participant Agent ID', 'Full Name',
       'CallerType', 'CLIENT NAME', 'COBRAEnrollIndicator', 'ConnID',
       'Cross Complete ID', 'DentalPlan', 'EmploymentStatus',
       'Inbound_CLIENT ID', 'MedicalPlan', 'MedicareEnrollIndicator',
       'nvcExportRule', 'Outbound_CLIENT_ID', 'PersonID', 'PrimarySubtopic',
       'PrimaryTopic', 'Recording Channel', 'Recording Logger',
       'Segment Duration', 'Skill', 'Universal ID'],
      dtype='object')

In [122]:
(concat_df_bmo_meta[concat_df_bmo_meta['Universal ID'].isin(['383103734','383119895','383104810','383101247','383097897','383137156','383129148','383098429','387255327'])]).columns

Index(['Path', 'File Name', 'Status', 'Segment ID', 'Segment Start Time ',
       'Segment Stop Time', 'Internal Segment Client Start Time',
       'Internal Segment Client Stop Time', 'Participant Station',
       'Segment UCID', 'Participant Trunk Group', 'Participant Trunk Number',
       'Segment Dialed Number', 'Participant Phone-Number',
       'Segment Call Direction Type ID', 'Participant Agent ID', 'Full Name',
       'CallerType', 'CLIENT NAME', 'COBRAEnrollIndicator', 'ConnID',
       'Cross Complete ID', 'DentalPlan', 'EmploymentStatus',
       'Inbound_CLIENT ID', 'MedicalPlan', 'MedicareEnrollIndicator',
       'nvcExportRule', 'Outbound_CLIENT_ID', 'PersonID', 'PrimarySubtopic',
       'PrimaryTopic', 'Recording Channel', 'Recording Logger',
       'Segment Duration', 'Skill', 'Universal ID'],
      dtype='object')

In [126]:
((concat_df_bmo_meta[concat_df_bmo_meta['Universal ID'].isin(['383103734','383119895','383104810','383101247','383097897','383137156','383129148','383098429','387255327'])])[['Universal ID','Segment Start Time ','Segment Stop Time','PrimarySubtopic',
       'PrimaryTopic']]).to_excel('seg_time.xlsx',index=False)

In [16]:
concat_df_bmo_trans.head(1)

,jobName,isRedacted,results,sub_bucket,transcripts,Date_jobName
0,6991817428750645949,True,{'transcripts': [{'transcript': 'Thank you for...,calldata/bmo/2021-08-02,Thank you for calling the benefit center. My n...,2021-08-16


In [17]:
concat_df_bmo_trans.head()

,jobName,isRedacted,results,sub_bucket,transcripts,Date_jobName
0,6991817428750645949,True,{'transcripts': [{'transcript': 'Thank you for...,calldata/bmo/2021-08-02,Thank you for calling the benefit center. My n...,2021-08-16
1,6991830133263897261,True,{'transcripts': [{'transcript': 'Hello? Hello?...,calldata/bmo/2021-08-02,Hello? Hello? No. Hello? I know. I know. I know.,2021-08-16
2,6991830468271346497,True,"{'transcripts': [{'transcript': ''}], 'items':...",calldata/bmo/2021-08-02,,2021-08-16
3,6991833839971669842,True,{'transcripts': [{'transcript': 'Thank you for...,calldata/bmo/2021-08-02,Thank you for calling Bmo Financial Group Bene...,2021-08-16
4,6991834252137536416,True,{'transcripts': [{'transcript': 'Thank you for...,calldata/bmo/2021-08-02,Thank you for calling Bmo Financial Group Bene...,2021-08-16


In [18]:
concat_df_bmo_trans.jobName.max()

'7011630336376378620'

In [15]:
# concat_df_bmo_trans = concat_df_bmo_trans[concat_df_bmo_trans['jobName'].astype(float).isin(segment_universal_ids['Segment ID'])]

In [20]:
concat_df_bmo_meta["Segment ID"].nunique()

59250

In [21]:
concat_df_bmo_meta["Segment ID"].nunique() + concat_df_bmo_meta["Segment ID"].isnull().sum()

59730

In [22]:
concat_df_bmo_meta["Segment Start Time "] = pd.to_datetime(concat_df_bmo_meta["Segment Start Time "])
concat_df_bmo_meta["Segment Stop Time"] = pd.to_datetime(concat_df_bmo_meta["Segment Stop Time"])

In [23]:
concat_df_bmo_meta["Segment Start Time "].min()

Timestamp('2020-08-04 12:21:28.233000')

In [24]:
concat_df_bmo_meta["Segment Start Time "].max()

Timestamp('2021-09-24 22:26:49.333000')

In [25]:
len(concat_df_bmo_trans)

69415

In [26]:
concat_df_bmo_trans.head(1)

,jobName,isRedacted,results,sub_bucket,transcripts,Date_jobName
0,6991817428750645949,True,{'transcripts': [{'transcript': 'Thank you for...,calldata/bmo/2021-08-02,Thank you for calling the benefit center. My n...,2021-08-16


In [27]:
concat_df_bmo_trans.shape

(69415, 6)

In [28]:
concat_df_bmo_trans.Date_jobName.unique()

array([datetime.date(2021, 8, 16), datetime.date(2021, 8, 18),
       datetime.date(2021, 8, 19), datetime.date(2021, 8, 20),
       datetime.date(2021, 8, 21), datetime.date(2021, 8, 24),
       datetime.date(2021, 8, 25), datetime.date(2021, 8, 26),
       datetime.date(2021, 8, 27), datetime.date(2021, 8, 28),
       datetime.date(2021, 8, 31), datetime.date(2021, 9, 1),
       datetime.date(2021, 7, 2), datetime.date(2021, 9, 2),
       datetime.date(2021, 9, 3), datetime.date(2021, 9, 4),
       datetime.date(2021, 9, 5), datetime.date(2021, 9, 6),
       datetime.date(2021, 9, 9), datetime.date(2021, 9, 10),
       datetime.date(2021, 9, 11), datetime.date(2021, 9, 12),
       datetime.date(2021, 9, 13), datetime.date(2021, 9, 14),
       datetime.date(2021, 9, 15), datetime.date(2021, 9, 16),
       datetime.date(2021, 9, 17), datetime.date(2021, 9, 18),
       datetime.date(2021, 9, 19), datetime.date(2021, 9, 20),
       datetime.date(2021, 9, 21), datetime.date(2021, 9, 22),


In [29]:
concat_df_bmo_trans["jobName"] = concat_df_bmo_trans["jobName"].astype('float')
concat_df_bmo_trans = concat_df_bmo_trans.rename(columns={"jobName":"Segment ID"})
concat_df_bmo_trans.head(1)

,Segment ID,isRedacted,results,sub_bucket,transcripts,Date_jobName
0,6.991817e+18,True,{'transcripts': [{'transcript': 'Thank you for...,calldata/bmo/2021-08-02,Thank you for calling the benefit center. My n...,2021-08-16


In [30]:
concat_df_bmo_trans.columns

Index(['Segment ID', 'isRedacted', 'results', 'sub_bucket', 'transcripts',
       'Date_jobName'],
      dtype='object')

In [31]:
useful_meta_cols = ['File Name','Segment ID','Universal ID',"Segment Start Time ","Segment Stop Time","Segment Call Direction Type ID",
                    "Segment Duration",'PrimarySubtopic','PrimaryTopic']

concat_df_bmo_trans = concat_df_bmo_trans.merge(concat_df_bmo_meta[useful_meta_cols],on="Segment ID",copy=False)

In [26]:
# concat_df_bmo_trans = concat_df_bmo_trans.merge(concat_df_bmo_trans.groupby('Universal ID',as_index=False)["Segment ID"].count()
#                         .rename(columns={'Segment ID':'Segment_Counts'}),
#                         on="Universal ID", copy=False)

In [32]:
concat_df_bmo_trans[concat_df_bmo_trans['Universal ID']== -1.0]['transcripts'].count()

3349

In [33]:
concat_df_bmo_trans['PrimaryTopic'].count()

18146

In [34]:
concat_df_bmo_trans.drop(concat_df_bmo_trans.index[concat_df_bmo_trans["Segment Call Direction Type ID"] == "Outgoing"], inplace=True)

In [35]:
concat_df_bmo_trans.count()

Segment ID                        52010
isRedacted                        41928
results                           52010
sub_bucket                        52010
transcripts                       52010
Date_jobName                      52010
File Name                         52010
Universal ID                      52010
Segment Start Time                52010
Segment Stop Time                 52010
Segment Call Direction Type ID    52010
Segment Duration                  52010
PrimarySubtopic                   16472
PrimaryTopic                      16472
dtype: int64

In [36]:
from datetime import timedelta
concat_df_bmo_trans = concat_df_bmo_trans[(concat_df_bmo_trans["Segment Stop Time"] - concat_df_bmo_trans["Segment Start Time "]) > timedelta(seconds=60)]

In [37]:
concat_df_bmo_trans.count()

Segment ID                        44147
isRedacted                        36055
results                           44147
sub_bucket                        44147
transcripts                       44147
Date_jobName                      44147
File Name                         44147
Universal ID                      44147
Segment Start Time                44147
Segment Stop Time                 44147
Segment Call Direction Type ID    44147
Segment Duration                  44147
PrimarySubtopic                   14531
PrimaryTopic                      14531
dtype: int64

In [38]:
len(concat_df_bmo_trans)

44147

In [39]:
bmo_df = concat_df_bmo_trans.copy()

In [40]:
bmo_df[bmo_df['Universal ID']==-1].shape

(2169, 14)

In [65]:
new_df.shape

(41978, 14)

In [89]:
new_df=bmo_df[~bmo_df['Universal ID'].isin([-1])]

In [52]:
new_df_notnull=new_df[~new_df['PrimarySubtopic'].isna()]
new_df_notnull.shape

(14531, 14)

In [53]:
new_df_notnull[new_df_notnull['Universal ID'].duplicated(keep=False)].head(6)

,Segment ID,isRedacted,results,sub_bucket,transcripts,Date_jobName,File Name,Universal ID,Segment Start Time,Segment Stop Time,Segment Call Direction Type ID,Segment Duration,PrimarySubtopic,PrimaryTopic
248,6.992994e+18,True,"{'transcripts': [{'transcript': 'new.'}], 'spe...",calldata/bmo/2021-08-05,new.,2021-08-16,NICE_6992994460243143133_Summed.wav,387474082.0,2021-08-05 17:10:01.003,2021-08-05 17:24:21.050,Incoming,00:14:20,Plan Provisions/Procedures,HW - Life Event/Qual Stat Chg
250,6.992994e+18,True,"{'transcripts': [{'transcript': 'Hello, this i...",calldata/meijer/2021-08-05,"Hello, this is I have the pleasure of assistan...",2021-08-07,NICE_6992994460243143133_Summed.wav,387474082.0,2021-08-05 17:10:01.003,2021-08-05 17:24:21.050,Incoming,00:14:20,Plan Provisions/Procedures,HW - Life Event/Qual Stat Chg
440,6.994424e+18,True,{'transcripts': [{'transcript': 'calling Bank ...,calldata/bmo/2021-08-09,"calling Bank of Montreal, your speaking to May...",2021-08-16,NICE_6994423640335920977_Summed.wav,387625731.0,2021-08-09 13:36:06.193,2021-08-09 13:43:05.460,Incoming,00:06:59,ROE,Payroll - Ongoing Processing
446,6.994425e+18,True,{'transcripts': [{'transcript': '[PII] [PII] [...,calldata/bmo/2021-08-09,"[PII] [PII] [PII] to continue in english, pres...",2021-08-16,NICE_6994425474286958893_Summed.wav,387625731.0,2021-08-09 13:43:05.460,2021-08-09 13:44:15.533,Incoming,00:01:10,ROE,Payroll - Ongoing Processing
491,6.994435e+18,True,{'transcripts': [{'transcript': 'how may I hel...,calldata/bmo/2021-08-09,"how may I help you? Hi, I have a manager on th...",2021-08-16,NICE_6994435296877162789_Summed.wav,387630840.0,2021-08-09 14:21:13.427,2021-08-09 14:35:32.620,Incoming,00:14:19,Processed Transaction,DC - Loans
493,6.994435e+18,True,"{'transcripts': [{'transcript': 'Good morning,...",calldata/meijer/2021-08-09,"Good morning, this is [PII]. Hi [PII], my name...",2021-08-10,NICE_6994435296877162789_Summed.wav,387630840.0,2021-08-09 14:21:13.427,2021-08-09 14:35:32.620,Incoming,00:14:19,Processed Transaction,DC - Loans


In [58]:
new_df_notnull=new_df_notnull[~new_df_notnull['transcripts'].duplicated()]

In [61]:
new_df_notnull[new_df_notnull['transcripts'].duplicated()].shape

(0, 14)

In [90]:
new_df_null=new_df[new_df['PrimarySubtopic'].isna()]

In [91]:
new_df_null.shape

(27447, 14)

In [92]:
df1 = pd.read_csv(r's3://adl-core-sagemaker-studio/external/Athena_output/Unsaved/2021/09/09/b944ea8e-837c-43f5-852c-db3a1ce40b3e.csv')

In [93]:
new_df_null.isna().sum()

Segment ID                            0
isRedacted                         4842
results                               0
sub_bucket                            0
transcripts                           0
Date_jobName                          0
File Name                             0
Universal ID                          0
Segment Start Time                    0
Segment Stop Time                     0
Segment Call Direction Type ID        0
Segment Duration                      0
PrimarySubtopic                   27447
PrimaryTopic                      27447
dtype: int64

In [94]:
df1 = df1.rename(columns={"call_unique_id":"Universal ID"})

In [95]:
useful_meta_cols = ['Universal ID','cspro_primary_topic','cspro_primary_subtopic']

new_df_null = new_df_null.merge(df1[useful_meta_cols],on="Universal ID",copy=False)

In [96]:
new_df_null.isna().sum()

Segment ID                           0
isRedacted                        3282
results                              0
sub_bucket                           0
transcripts                          0
Date_jobName                         0
File Name                            0
Universal ID                         0
Segment Start Time                   0
Segment Stop Time                    0
Segment Call Direction Type ID       0
Segment Duration                     0
PrimarySubtopic                   6101
PrimaryTopic                      6101
cspro_primary_topic                  0
cspro_primary_subtopic               0
dtype: int64

In [98]:
new_df_null.drop(columns = ['PrimarySubtopic','PrimaryTopic'],inplace = True)
new_df_null = new_df_null.rename(columns={"cspro_primary_topic":"PrimaryTopic",
                                             "cspro_primary_subtopic":"PrimarySubtopic"})

In [99]:
new_df_null = new_df_null[new_df_null['transcripts'].notnull()]

In [101]:
new_df_null.shape

(6101, 14)

In [102]:
new_df_notnull.count()

Segment ID                        12742
isRedacted                         9731
results                           12742
sub_bucket                        12742
transcripts                       12742
Date_jobName                      12742
File Name                         12742
Universal ID                      12742
Segment Start Time                12742
Segment Stop Time                 12742
Segment Call Direction Type ID    12742
Segment Duration                  12742
PrimarySubtopic                   12742
PrimaryTopic                      12742
dtype: int64

In [103]:
df_with_topics = pd.concat([new_df_null,new_df_notnull])
df_with_topics =df_with_topics[df_with_topics['transcripts'].notnull()]
df_with_topics.count()

Segment ID                        18843
isRedacted                        12550
results                           18843
sub_bucket                        18843
transcripts                       18843
Date_jobName                      18843
File Name                         18843
Universal ID                      18843
Segment Start Time                18843
Segment Stop Time                 18843
Segment Call Direction Type ID    18843
Segment Duration                  18843
PrimaryTopic                      18843
PrimarySubtopic                   18843
dtype: int64

In [104]:
df_with_topics.isna().sum()

Segment ID                           0
isRedacted                        6293
results                              0
sub_bucket                           0
transcripts                          0
Date_jobName                         0
File Name                            0
Universal ID                         0
Segment Start Time                   0
Segment Stop Time                    0
Segment Call Direction Type ID       0
Segment Duration                     0
PrimaryTopic                         0
PrimarySubtopic                      0
dtype: int64

In [110]:
df_with_topics[df_with_topics['transcripts'].duplicated(keep=False)]

,Segment ID,isRedacted,results,sub_bucket,transcripts,Date_jobName,File Name,Universal ID,Segment Start Time,Segment Stop Time,Segment Call Direction Type ID,Segment Duration,PrimaryTopic,PrimarySubtopic
150,6.993002e+18,True,"{'transcripts': [{'transcript': ''}], 'items':...",calldata/bmo/2021-08-05,,2021-08-16,NICE_6993002139304933694_Summed.wav,387476532.0,2021-08-05 17:39:49.153,2021-08-05 17:41:21.373,Incoming,00:01:32,HW,Spending Accounts
190,6.993371e+18,True,"{'transcripts': [{'transcript': ''}], 'items':...",calldata/bmo/2021-08-06,,2021-08-16,NICE_6993371390528268986_Summed.wav,387560640.0,2021-08-06 17:32:43.897,2021-08-06 17:35:20.963,Incoming,00:02:37,Automatically generated due to unexpected syst...,pay_inq
242,6.994435e+18,True,{'transcripts': [{'transcript': 'Thank you for...,calldata/bmo/2021-08-09,"Thank you for calling Bmo HR See my name is, h...",2021-08-16,NICE_6994434661222002002_Summed.wav,387632682.0,2021-08-09 14:18:52.337,2021-08-09 14:21:33.527,Incoming,00:02:41,WFA - Job & Comp Change,Policy/Procedures
243,6.994435e+18,True,{'transcripts': [{'transcript': 'thank you for...,calldata/bmo/2021-08-09,"thank you for calling charge with Richard, can...",2021-08-16,NICE_6994434661222002002_Summed.wav,387632682.0,2021-08-09 14:18:52.337,2021-08-09 14:21:33.527,Incoming,00:02:41,WFA - Job & Comp Change,Policy/Procedures
244,6.994435e+18,True,{'transcripts': [{'transcript': 'this is [PII]...,calldata/meijer/2021-08-09,this is [PII] who do I have the pleasure of sp...,2021-08-10,NICE_6994434661222002002_Summed.wav,387632682.0,2021-08-09 14:18:52.337,2021-08-09 14:21:33.527,Incoming,00:02:41,WFA - Job & Comp Change,Policy/Procedures
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44610,6.988189e+18,True,"{'transcripts': [{'transcript': '[PII], are yo...",calldata/meijer/2021-07-23,"[PII], are you there? Yes, I'm here. Alright, ...",2021-07-24,NICE_6988188911190218247_Summed.wav,386618481.0,2021-07-23 18:22:02.003,2021-07-23 18:24:55.867,Incoming,00:02:53,CM - External Referral,Carrier - Dental
44612,6.988189e+18,True,{'transcripts': [{'transcript': 'thank you for...,calldata/meijer/2021-07-23,thank you for calling my name is and may I hav...,2021-07-24,NICE_6988188911190218247_Summed.wav,386618481.0,2021-07-23 18:22:02.003,2021-07-23 18:24:55.867,Incoming,00:02:53,CM - External Referral,Carrier - Dental
45936,6.989966e+18,True,{'transcripts': [{'transcript': 'thank you for...,calldata/meijer/2021-07-28,thank you for calling Myers. My name is [PII]....,2021-07-29,NICE_6989965507232342534_Summed.wav,386867540.0,2021-07-28 13:16:14.003,2021-07-28 13:28:11.097,Incoming,00:11:57,DB - Payments,Withholding Changes/Questions
45938,6.989966e+18,True,"{'transcripts': [{'transcript': 'Good morning,...",calldata/meijer/2021-07-28,"Good morning, this is [PII], how can I help yo...",2021-07-29,NICE_6989965507232342534_Summed.wav,386867540.0,2021-07-28 13:16:14.003,2021-07-28 13:28:11.097,Incoming,00:11:57,DB - Payments,Withholding Changes/Questions


In [111]:
df_with_topics["Universal ID"].nunique()

16428

In [112]:
df_with_topics.to_excel("all_bmo_meijer_28sep2021.xlsx",index=False)

to_csv=pd.read_excel('s3://adl-core-sagemaker-studio/external/User/aws_call_data/BMO_Meijer_single_leg_noNull_17Sep2021.xlsx')

to_csv.shape

to_csv.to_csv('BMO_Meijer_single_leg_noNull_17Sep2021.csv')

import boto3
import os
boto3.Session().resource('s3').Bucket('adl-core-sagemaker-studio').Object(os.path.join('external/User/aws_call_data/', 'BMO_Meijer_single_leg_noNull_17Sep2021.csv')).upload_file('BMO_Meijer_single_leg_noNull_17Sep2021.csv')


In [70]:
import boto3
import os
boto3.Session().resource('s3').Bucket('adl-core-sagemaker-studio').Object(os.path.join('external/User/rhp/Aws_data/All_data/', 'all_bmo_meijer_28sep2021.xlsx')).upload_file('all_bmo_meijer_28sep2021.xlsx')


In [40]:
s3://adl-core-sagemaker-studio/external/User/rhp/Aws_data/All_data/

,Segment ID,isRedacted,results,sub_bucket,transcripts,Date_jobName,File Name,Universal ID,Segment Start Time,Segment Stop Time,Segment Call Direction Type ID,Segment Duration,PrimarySubtopic,PrimaryTopic
0,6.991817e+18,True,{'transcripts': [{'transcript': 'Thank you for...,calldata/bmo/2021-08-02,Thank you for calling the benefit center. My n...,2021-08-16,NICE_6991817428750645949_Summed.wav,-1.0,2021-08-02 13:02:32.090,2021-08-02 13:19:56.160,Incoming,00:17:24,NaN,NaN


In [41]:
len(bmo_df)

36039

In [94]:
# one_segments_df = bmo_df.query("Segment_Counts == 1")
# multi_segments_df = bmo_df.query("Segment_Counts > 1")

In [96]:
# multi_segments_df.head(1)

In [97]:
# print(f"Total no. of trancripts Single segment calls: {len(one_segments_df)}")
# print("Total no. of unique calls with multiple segments: {}".format(multi_segments_df["Universal ID"].nunique()))
# print(f"The segments in multi-segment calls ranges from {multi_segments_df.Segment_Counts.min()} to {multi_segments_df.Segment_Counts.max()}")

In [98]:
# print(f"Total no. of trancripts Single segment calls: {len(one_segments_df)}")
# print(f"Total no. of trancripts Multiple segment calls: {len(multi_segments_df)}")

In [42]:
bmo_df.to_excel("all_bmo_meijer_data_17sep2021.xlsx",index=False)

In [43]:
bmo_df.isna().sum()

Segment ID                            0
isRedacted                         8092
results                               0
sub_bucket                            0
transcripts                           0
Date_jobName                          0
File Name                             0
Universal ID                          0
Segment Start Time                    0
Segment Stop Time                     0
Segment Call Direction Type ID        0
Segment Duration                      0
PrimarySubtopic                   24146
PrimaryTopic                      24146
dtype: int64

In [6]:
bmo_df.shape

(0, 14)

In [52]:
bmo_df=bmo_df[bmo_df['transcripts']!='']

In [53]:
bmo_df[bmo_df['Universal ID']==387738535.0]

,Segment ID,isRedacted,results,sub_bucket,transcripts,Date_jobName,File Name,Universal ID,Segment Start Time,Segment Stop Time,Segment Call Direction Type ID,Segment Duration,PrimarySubtopic,PrimaryTopic
1121,6.994809e+18,True,{'transcripts': [{'transcript': 'Good morning....,calldata/meijer/2021-08-10,Good morning. This is [PII]. Uh Yes. My name i...,2021-08-11,NICE_6994809474427991867_Summed.wav,387738535.0,2021-08-10 14:33:13.993,2021-08-10 14:34:15.813,Incoming,00:01:01,Cashout vs. Rollover,CM - Retirement
29074,6.994808e+18,True,{'transcripts': [{'transcript': 'thank you for...,calldata/meijer/2021-08-10,thank you for calling the benefits center. My ...,2021-08-11,NICE_6994808439831608077_Summed.wav,387738535.0,2021-08-10 14:29:22.000,2021-08-10 14:33:14.133,Incoming,00:03:52,Cashout vs. Rollover,CM - Retirement
29083,6.994809e+18,True,{'transcripts': [{'transcript': 'Good morning....,calldata/meijer/2021-08-10,Good morning. This is uh yes. My name is [PII]...,2021-08-11,NICE_6994809483508661205_Summed.wav,387738535.0,2021-08-10 14:33:14.133,2021-08-10 14:34:15.950,Incoming,00:01:01,Cashout vs. Rollover,CM - Retirement


In [54]:
bmo_df=bmo_df[~bmo_df['Segment ID'].isin(segment_universal_ids['Segment ID'])]
bmo_df.shape
bmo_df.to_csv("all_bmo_meijer_100call_rmv.csv",index=False)

In [40]:
bmo_df[bmo_df['Segment ID'].isin(segment_universal_ids['Segment ID'])].to_csv("bmo_meijer_data_to_be_predicted.csv",index=False)

In [1]:
import pandas as pd
df=pd.read_csv('all_bmo_meijer_100call_rmv.csv',low_memory=False)

In [2]:
df

,Segment ID,isRedacted,results,sub_bucket,transcripts,Date_jobName,File Name,Universal ID,Segment Start Time,Segment Stop Time,Segment Call Direction Type ID,Segment Duration,PrimarySubtopic,PrimaryTopic
0,6.991817e+18,True,"{'transcripts': [{'transcript': ""Thank you for...",calldata/bmo/2021-08-02,Thank you for calling the benefit center. My n...,2021-08-16,NICE_6991817428750645949_Summed.wav,-1.0,2021-08-02 13:02:32.090,2021-08-02 13:19:56.160,Incoming,00:17:24,NaN,NaN
1,6.992192e+18,True,"{'transcripts': [{'transcript': ""Thank you for...",calldata/bmo/2021-08-03,Thank you for calling dependent verification s...,2021-08-16,NICE_6992191851114594948_Summed.wav,-1.0,2021-08-03 13:15:29.567,2021-08-03 13:24:30.727,Incoming,00:09:01,NaN,NaN
2,6.992249e+18,True,"{'transcripts': [{'transcript': ""Hi, thank you...",calldata/bmo/2021-08-03,"Hi, thank you calling BMO HR See you are speak...",2021-08-16,NICE_6992248660307291849_Summed.wav,-1.0,2021-08-03 16:55:57.657,2021-08-03 16:58:08.330,Incoming,00:02:10,NaN,NaN
3,6.992561e+18,True,"{'transcripts': [{'transcript': ""Thank you for...",calldata/bmo/2021-08-04,Thank you for calling dependent verification? ...,2021-08-16,NICE_6992561338221399089_Summed.wav,-1.0,2021-08-04 13:09:24.333,2021-08-04 13:14:10.013,Incoming,00:04:45,NaN,NaN
4,6.992564e+18,True,"{'transcripts': [{'transcript': ""Hi, thank you...",calldata/bmo/2021-08-04,"Hi, thank you for calling Charge you are speak...",2021-08-16,NICE_6992563666093675733_Summed.wav,-1.0,2021-08-04 13:18:20.947,2021-08-04 13:23:54.830,Incoming,00:05:33,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21353,7.000127e+18,True,"{'transcripts': [{'transcript': ""good afternoo...",calldata/meijer/2021-08-24,"good afternoon, this is [PII] how may I help y...",2021-08-25,NICE_7000126751314620475_Summed.wav,388759067.0,2021-08-24 22:27:02.827,2021-08-24 22:37:27.813,Incoming,00:10:24,NaN,NaN
21354,7.000127e+18,True,"{'transcripts': [{'transcript': ""Hello? Hello?...",calldata/meijer/2021-08-24,"Hello? Hello? Hello, I can hear you for callin...",2021-08-25,NICE_7000127065449119431_Summed.wav,388759148.0,2021-08-24 22:28:10.557,2021-08-24 22:30:32.973,Incoming,00:02:22,NaN,NaN
21355,7.000128e+18,True,"{'transcripts': [{'transcript': ""Alright, so t...",calldata/meijer/2021-08-24,"Alright, so that's been successfully set up fo...",2021-08-25,NICE_7000127696809311972_Summed.wav,388759148.0,2021-08-24 22:30:37.763,2021-08-24 22:39:05.407,Incoming,00:08:27,Initiate Commencement,CM - Retirement
21356,7.000128e+18,True,"{'transcripts': [{'transcript': ""continue to h...",calldata/meijer/2021-08-24,continue to hold. Did you know the easiest and...,2021-08-25,NICE_7000127700502393271_Summed.wav,388758648.0,2021-08-24 22:30:38.390,2021-08-24 22:47:00.840,Incoming,00:16:22,NaN,NaN


In [5]:
bmo_df=df[df['Segment ID'].isin(segment_universal_ids['Segment ID'])]
bmo_df.shape

(0, 14)